# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------

### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [ ]:
df_em = pd.read_csv(path + "/df_basic_true.csv")
df_em

,기관명,기관주소,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),37.240316,127.214491
...,...,...,...,...
521,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",37.393404,126.924477
522,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",37.358645,126.947360
523,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),35.185413,129.121459
524,흑룡의원,인천광역시 옹진군 백령면 백령로 831,37.959524,124.665499


In [ ]:
info_em = pd.read_csv(path + "/응급실 정보.csv")
info_em

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491
...,...,...,...,...,...,...,...
520,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",지역응급의료센터,031-467-9717,031-467-9119,37.393404,126.924477
521,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",지역응급의료센터,031-389-3000,031-389-3119,37.358645,126.947360
522,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),응급실운영신고기관,051-709-3000,051-709-3119,35.185413,129.121459
523,흑룡의원,인천광역시 옹진군 백령면 백령로 831,응급실운영신고기관,032-837-6873,032-837-3153,37.959524,124.665499


In [ ]:
lag_em = info_em['위도']
lan_em = info_em['경도']

## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')
* kt : 37.358877 127.114863
* 분당 서울대병원 37.352028 127.123245
* 미소병원 37.350639 127.108367


In [ ]:
print(haversine((37.358877, 127.114863) , (37.352028, 127.123245),unit='km'))
print(haversine((37.358877, 127.114863) , (37.350639, 127.108367),unit='km'))

1.0624857198851487
1.0810983139347548


#### 2) 가장 가까운 응급실 3곳 추천하기1

In [ ]:
x = (37.358877, 127.114863)
def recom_em(x):
    distance = []
    for i in range(len(info_em)):
        lag_em = info_em['위도']
        lan_em = info_em['경도']
        dist = haversine(x, (lag_em[i], lan_em[i]), unit='km')
        distance.append((dist, info_em['병원이름'][i]))

    # 거리를 기준으로 정렬
    distance.sort(key=lambda x: x[0])

    # 가장 가까운 3개의 병원 출력
    for i in range(3):
        print(distance[i])

In [ ]:
recom_em(x)

(1.14167026531942, '분당서울대학교병원')
(3.2742595237709855, '대진의료재단분당제생병원')
(4.726119154441283, '국군수도병원')


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
* 응급실 정보에서 해당 구간을 먼저 조회한 후
* 거리 계산

In [ ]:
x = (37.358877, 127.114863)
def recom_em(x):
    a = 0.5
    x_lat, x_lon = int(x[0]), int(x[1])
    x_lat_min, x_lon_min = x_lat - a, x_lon - a
    x_lat_max, x_lon_max = x_lat + a, x_lon + a
    lat_em, lon_em = info_em['위도'], info_em['경도']
    filtered_hospitals = info_em[(lat_em >= x_lat_min) & (lat_em <= x_lat_max) & (lon_em >= x_lon_min) & (lon_em <= x_lon_max)]

    distance = []

    for i in range(len(filtered_hospitals)):
        lat_em_filter, lon_em_filter = filtered_hospitals.iloc[i]['위도'], filtered_hospitals.iloc[i]['경도']
        dist = haversine(x, (lat_em_filter, lon_em_filter), unit='km')
        distance.append((dist, filtered_hospitals.iloc[i]['병원이름']))

    # 거리를 기준으로 정렬
    distance.sort(key=lambda x: x[0])

    # 가장 가까운 3개의 병원 출력
    for i in range(min(3, len(distance))):
        print(distance[i])

In [ ]:
x = (37.358877, 127.114863)
recom_em(x)

(1.14167026531942, '분당서울대학교병원')
(3.2742595237709855, '대진의료재단분당제생병원')
(4.726119154441283, '국군수도병원')


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [ ]:
cilent_id = ''
cilent_key = ''

#### 2) 함수 생성

In [ ]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": cilent_id,
        "X-NCP-APIGW-API-KEY": cilent_key,
    }
    params = {
        "start": f"{start_lng}, {start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng}, {dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)
    response = response.json()
    dist = response['route']['trafast'][0]['summary']['distance']  # m(미터)

    return dist

* 테스트

In [ ]:
test = pd.read_excel(path + "/audio_location.xlsx")
test

,filename,위도,경도,Unnamed: 3,Unnamed: 4
0,audio1.mp3,37.358618,127.115036,NaN,NaN
1,audio2.mp3,36.815571,127.128844,NaN,NaN
2,audio3.mp3,37.538435,126.989828,NaN,NaN
3,audio4.mp3,35.185745,129.076541,NaN,NaN
4,audio5.mp3,36.503544,127.252941,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,


In [ ]:
get_dist(37.358877, 127.114863, 37.352028, 127.123245, cilent_id, cilent_key)

2461

#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [ ]:
x = (37.8543571, 128.0248129)
x_lat, x_lon =  x[0], x[1]
a = 0.1
x_lat_min, x_lon_min = x_lat - a, x_lon - a
x_lat_max, x_lon_max = x_lat + a, x_lon + a
lat_em, lon_em = info_em['위도'], info_em['경도']
filtered_hospitals = info_em[(lat_em >= x_lat_min) & (lat_em <= x_lat_max) & (lon_em >= x_lon_min) & (lon_em <= x_lon_max)]
len(filtered_hospitals)

0

In [ ]:
def recom_em(x):
    x_lat, x_lon =  x[0], x[1]
    a = 0.1
    x_lat_min, x_lon_min = x_lat - a, x_lon - a
    x_lat_max, x_lon_max = x_lat + a, x_lon + a
    lat_em, lon_em = info_em['위도'], info_em['경도']
    filtered_hospitals = info_em[(lat_em >= x_lat_min) & (lat_em <= x_lat_max) & (lon_em >= x_lon_min) & (lon_em <= x_lon_max)]

    distance = []
    #api 불러오기
    def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": cilent_id,
            "X-NCP-APIGW-API-KEY": cilent_key,
        }
        params = {
            "start": f"{start_lng}, {start_lat}",  # 출발지 (경도, 위도)
            "goal": f"{dest_lng}, {dest_lat}",    # 목적지 (경도, 위도)
            "option": "trafast"  # 실시간 빠른 길 옵션
        }

        # 요청하고, 답변 받아오기
        response = requests.get(url, headers=headers, params=params)
        response = response.json()
        dist = response['route']['trafast'][0]['summary']['distance']  # m(미터)

        return dist

    for i in range(len(filtered_hospitals)):
        lat_em_filter, lon_em_filter = filtered_hospitals.iloc[i]['위도'], filtered_hospitals.iloc[i]['경도']
        dist = get_dist(x_lat, x_lon, lat_em_filter, lon_em_filter, cilent_id, cilent_key)
        distance.append((dist, filtered_hospitals.iloc[i]['병원이름']))

    # 거리를 기준으로 정렬
    distance.sort(key=lambda x: x[0])

    # 가장 가까운 3개의 병원 출력
    for i in range(min(3, len(distance))):
        print(distance[i])

# 만약 없다면 0.1보다 늘려서 재탐색

In [ ]:
x = (37.358877, 127.114863)
recom_em(x)

(2778, '분당서울대학교병원')
(4377, '대진의료재단분당제생병원')
(6125, '국군수도병원')


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import requests

def recom_em(x):
    x_lat, x_lon = x[0], x[1]

    a = 0.1  # 초기 검색 범위
    filtered_hospitals = None

    # 최소 3개의 병원이 필터링될 때까지 검색 범위 확장
    while filtered_hospitals is None or len(filtered_hospitals) < 3:
        x_lat_min, x_lon_min = x_lat - a, x_lon - a
        x_lat_max, x_lon_max = x_lat + a, x_lon + a

        # 위도와 경도 조건으로 병원 필터링
        lat_em, lon_em = info_em['위도'], info_em['경도']
        filtered_hospitals = info_em[
            (lat_em >= x_lat_min) & (lat_em <= x_lat_max) &
            (lon_em >= x_lon_min) & (lon_em <= x_lon_max)
        ]

        # 범위 확장
        a += 0.05

    # API 호출 함수
    def get_dist(hospital):
        lat_em_filter, lon_em_filter = hospital['위도'], hospital['경도']
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": cilent_id,
            "X-NCP-APIGW-API-KEY": cilent_key,
        }
        params = {
            "start": f"{x_lon}, {x_lat}",  # 출발지 (경도, 위도)
            "goal": f"{lon_em_filter}, {lat_em_filter}",  # 목적지 (경도, 위도)
            "option": "trafast"  # 실시간 빠른 길 옵션
        }
        response = requests.get(url, headers=headers, params=params).json()
        dist = response['route']['trafast'][0]['summary']['distance']  # 거리 (미터)
        return (dist, hospital['병원이름'], hospital['전화번호 3'], hospital['전화번호 1'])
        # 전화번호 3가 응급의료시설번호고, 전화번호 1이 병원번호

    # 병렬 처리로 API 요청 가속화
    with ThreadPoolExecutor() as executor:
         results = list(executor.map(get_dist, filtered_hospitals.to_dict('records')))

    # 거리 기준 정렬
    results.sort(key=lambda x: x[0])

    # 가장 가까운 3개의 병원 출력
    for i in range(min(3, len(results))):
        print(results[i])
        print(a)

In [ ]:
x = (37.8543571, 128.0248129)

In [ ]:
recom_em(x)

(22947, '홍천아산병원', '033-430-5119', '033-430-5211')
0.35
(41379, '의료법인성심의료재단양구성심병원', '033-480-8816', '033-480-8803')
0.35
(49881, '강원대학교병원', '033-258-2245', '033-258-2000')
0.35
